# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,shitanjing,39.23,106.34,16.17,56,84,1.20,CN,1601430260
1,yerbogachen,61.28,108.01,1.43,80,0,2.03,RU,1601430261
2,chengam,12.30,78.80,22.27,88,100,3.00,IN,1601430261
3,clyde river,41.30,-82.98,12.78,76,52,0.45,US,1601430262
4,nome,64.50,-165.41,6.11,60,90,3.60,US,1601430262


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,39.23,106.34
1,61.28,108.01
2,12.30,78.80
3,41.30,-82.98
4,64.50,-165.41


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0    56
1    80
2    88
3    76
4    60
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 26


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,yeppoon,-23.13,150.73,25.00,36,0,5.10,AU,1601430196
22,carnarvon,-24.87,113.63,22.00,46,0,6.70,AU,1601430275
31,palaikastron,35.20,26.25,23.33,52,0,6.71,GR,1601430280
43,gat,31.61,34.76,23.89,86,0,1.88,IL,1601430290
45,atuona,-9.80,-139.03,25.74,75,0,8.23,PF,1601430291
66,turayf,31.67,38.66,21.00,40,0,4.10,SA,1601430303
118,buraydah,26.33,43.98,22.00,35,0,3.10,SA,1601430337
139,morondava,-20.28,44.28,22.99,66,0,2.88,MG,1601430352
145,lahij,13.17,44.58,25.90,74,0,1.25,YE,1601430356
147,ambilobe,-13.20,49.05,20.79,91,0,1.18,MG,1601430358


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
14,yeppoon,-23.13,150.73,AU,
22,carnarvon,-24.87,113.63,AU,
31,palaikastron,35.20,26.25,GR,
43,gat,31.61,34.76,IL,
45,atuona,-9.80,-139.03,PF,
66,turayf,31.67,38.66,SA,
118,buraydah,26.33,43.98,SA,
139,morondava,-20.28,44.28,MG,
145,lahij,13.17,44.58,YE,
147,ambilobe,-13.20,49.05,MG,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for yeppoon
--------------------------
Finding nearest hotel for carnarvon
--------------------------
Finding nearest hotel for palaikastron
--------------------------
Finding nearest hotel for gat
--------------------------
There are no Hotels for gat, moving on to the next city....

Finding nearest hotel for atuona
--------------------------
Finding nearest hotel for turayf
--------------------------
Finding nearest hotel for buraydah
--------------------------
Finding nearest hotel for morondava
--------------------------
Finding nearest hotel for lahij
--------------------------
There are no Hotels for lahij, moving on to the next city....

Finding nearest hotel for ambilobe
--------------------------
Finding nearest hotel for bushehr
--------------------------
Finding nearest hotel for nouadhibou
--------------------------
Finding nearest hotel for enumclaw
--------------------------
Finding nearest hotel for waddan
-----------------

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
14,yeppoon,-23.13,150.73,AU,Echelon Apartments
22,carnarvon,-24.87,113.63,AU,Carnarvon Motel
31,palaikastron,35.20,26.25,GR,Hotel Kastri Village
45,atuona,-9.80,-139.03,PF,Hôtel Hiva Oa Hanakee Lodge
66,turayf,31.67,38.66,SA,Golden Dune Hotel
118,buraydah,26.33,43.98,SA,Mövenpick Hotel Qassim
139,morondava,-20.28,44.28,MG,Palissandre Cote Ouest resort & SPA
147,ambilobe,-13.20,49.05,MG,Hotel National
148,bushehr,28.97,50.84,IR,Sky 2 hotel
169,nouadhibou,20.93,-17.03,MR,Free Zone Hotel Nouadhibou


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))